# Analisando Pacientes e Riscos

## imports

In [1]:
import pandas as pd

## Leitura das Bases

### Procedimentos com Risco

In [2]:
df_risk = pd.read_csv('data/procedimentos/dev/procedimentos_encontrados_no_risco_cirurgico.csv')
set_risk = set(df_risk['cod_procedimento'])

### Com Lúpus

In [5]:
num_algoritmo = 1

df_lupus = pd.read_parquet(f'data/results/df_alg{num_algoritmo}.parquet')
df_lupus_bpai = pd.read_parquet(f'data/results/df_bpai_alg{num_algoritmo}.parquet')
df_lupus_aih = pd.read_parquet(f'data/results/df_aih_alg{num_algoritmo}.parquet')
df_lupus_apac = pd.read_parquet(f'data/results/df_apac_alg{num_algoritmo}.parquet')

### Sem Lúpus

In [29]:
df_bpai = pd.read_parquet(f'data/BPAI_Lupus_L93M32N08_todos_cids.parquet')
df_aih = pd.read_parquet(f'data/AIH_Lupus_L93M32N08_todos_cids.parquet')
df_apac = pd.read_parquet(f'data/APAC_Lupus_L93M32N08_todos_cids.parquet')

In [ ]:
def get_cols_procedimentos(base_name):
      dict_col = {'BPAI':['co_procedimento_realizado'],
                  'AIH':['co_procedimento_principal'], # --> AIH nãm tem cod_procedimento_secundario
                  'APAC':['co_procedimento_principal', 'co_procedimento_secundario'],
                  'UNION':['co_procedimento_realizado', # BPAI
                           'co_procedimento_principal', # APAC
                           'co_procedimento_secundario', # APAC
                           'co_procedimento_aih']} # AIH
      return dict_col[base_name]

def get_risk(df, cols, set_risk):
    """ Calcula quantos pacientes possuem um procedimento de risco """
    if len(cols) > 1: # Se tiver mais de uma coluna de procedimentos
        cols_risk = []
        for idx, col in enumerate(cols): # Itera nas colunas
            col_name = f'risk_procedimento_{idx}'
            df[col_name] = df[col].isin(set_risk)
            cols_risk.append(col_name)

        df['risk_filter_col'] = df[cols_risk].sum(axis=1) # Conta quantas colnuas possuem True
        
        # Filtra as que tiveram pelo menos 1
        num_pacientes_risk = df[df['risk_filter_col'] > 0]['id_paciente'].nunique()

    else: # Se for só uma coluna, fica mais fácil
        num_pacientes_risk = df[df[cols[0]].isin(set_risk)]['id_paciente'].nunique()
    return num_pacientes_risk

def get_percent(numerador, denominador):
    return round(numerador*100/denominador, 2)

def analisa_riscos(df, df_lupus, base_name):
    cols_procedimento = get_cols_procedimentos(base_name)

    num_total_pacientes = df['id_paciente'].nunique()
    print(f' - Pacientes na {base_name}:', num_total_pacientes)
    
    num_pacientes_risk = get_risk(df, cols_procedimento, set_risk)
    print(f' - Pacientes na {base_name} com algum risco: '+
            f'{num_pacientes_risk} ' +
            f'({get_percent(num_pacientes_risk, num_total_pacientes)}%)')

    num_pacientes_lupus = df_lupus['id_paciente'].nunique()
    print(f' - Pacientes com Lúpus na {base_name}: {num_pacientes_lupus} '+
          f'({get_percent(num_pacientes_lupus, num_total_pacientes)})%')

    num_lupus_risk = get_risk(df_lupus, cols_procedimento, set_risk)

    print(f' - Pacientes com Lúpus e algum risco na {base_name}: {num_lupus_risk} ' +
            f'({get_percent(num_lupus_risk, num_pacientes_lupus)}% de Lúpus) e ' +
            f'({get_percent(num_lupus_risk, num_total_pacientes)}% da {base_name})')
    print()

analisa_riscos(df_bpai, df_lupus_bpai, 'BPAI')
analisa_riscos(df_aih, df_lupus_aih, 'AIH')
analisa_riscos(df_apac, df_lupus_apac, 'APAC')

 - Pacientes na BPAI: 46967
 - Pacientes na BPAI com algum risco: 33049 (70.37%)
 - Pacientes com Lúpus na BPAI: 5934 (12.63)%
 - Pacientes com Lúpus e algum risco na BPAI: 4940 (83.25% de Lúpus) e (10.52% da BPAI)

 - Pacientes na AIH: 48663
 - Pacientes na AIH com algum risco: 6906 (14.19%)
 - Pacientes com Lúpus na AIH: 8566 (17.6)%
 - Pacientes com Lúpus e algum risco na AIH: 1323 (15.44% de Lúpus) e (2.72% da AIH)

 - Pacientes na APAC: 31948
 - Pacientes na APAC com algum risco: 14418 (45.13%)
 - Pacientes com Lúpus na APAC: 6122 (19.16)%
 - Pacientes com Lúpus e algum risco na APAC: 1389 (22.69% de Lúpus) e (4.35% da APAC)



# Base Unificada
BPAI + AIH + APAC

In [50]:
df_alg = pd.read_parquet(f'data/results/df_alg{num_algoritmo}.parquet')

In [61]:
cols_union = ['id_paciente']
df_aih.rename(columns={'co_procedimento_principal':'co_procedimento_aih'})
df_union = pd.concat([df_bpai[cols_union+get_cols_procedimentos('BPAI')],
                      df_apac[cols_union+get_cols_procedimentos('APAC')],
                      df_aih.rename(columns={'co_procedimento_principal':'co_procedimento_aih'})\
                      [cols_union+['co_procedimento_aih']]])

In [69]:
cols_union = ['id_paciente']
df_lupus_aih.rename(columns={'co_procedimento_principal':'co_procedimento_aih'})
df_lupus_union = pd.concat([df_lupus_bpai[cols_union+get_cols_procedimentos('BPAI')],
                      df_lupus_apac[cols_union+get_cols_procedimentos('APAC')],
                      df_lupus_aih.rename(columns={'co_procedimento_principal':'co_procedimento_aih'})\
                      [cols_union+['co_procedimento_aih']]])

In [70]:
analisa_riscos(df_union, df_lupus_union, 'UNION')

 - Pacientes na UNION: 71874
 - Pacientes na UNION com algum risco: 43032 (59.87%)
 - Pacientes com Lúpus na UNION: 8987 (12.5)%
 - Pacientes com Lúpus e algum risco na UNION: 5695 (63.37% de Lúpus) e (7.92% da UNION)

